In [1]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-06 14:08:43--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-06 14:08:44 (55.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [10]:
import minsearch

In [3]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-06-06 14:14:22--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.006s  

2025-06-06 14:14:22 (100 MB/s) - ‘documents.json’ saved [658332/658332]



In [5]:
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [12]:
q = "the course has already started, can I still enroll?"

In [13]:
index.fit(documents)

In [14]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [15]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [16]:
from openai import OpenAI

In [17]:
client = OpenAI()

In [18]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [21]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on several factors, such as the institution offering the course, the type of course, and their specific policies on late enrollment. Here are some general steps you can take:\n\n1. **Check the Institution's Policy**: Look up the enrollment policies of the institution offering the course. Some institutions allow late enrollments within a certain period after the course start date.\n\n2. **Contact the Instructor or Administration**: Reach out directly to the course instructor or the admissions office. They may be able to grant you permission to enroll late.\n\n3. **Understand the Implications**: Be aware of any challenges related to enrolling late, such as catching up on missed material, deadlines, and potential grade impacts.\n\n4. **Consider Alternative Options**: If late enrollment is not possible, you might consider auditing the course, if available, or enrolling in a future offering of the course.\n\nMake sur

In [32]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [33]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [36]:
prompt = prompt_template.format(question=q, context=context).strip()

In [37]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [38]:
response.choices[0].message.content

"Yes, even if the course has already started, you can still enroll and are eligible to submit the homework. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute."